## demo van het valideren van dcat beschrijvingen op basis van de DCAT-AP-NL 3.0 shacl files

Dit is een jupyter notebook wat laat zien hoe pyshacl gebruikt kan worden om validaties te doen.

Om dit notebook uit te kunnen voeren heb je nodig:

- een python installatie met:
    - jupyter notebook
    - rdflib
    - pyshacl
    - pandas
    

In [1]:
# helper functie om te valideren met pyshacl
from pyshacl import validate
def do_validate(data_graph, sg):
    r = validate(data_graph,
      shacl_graph=sg,
      ont_graph=None,
      inference='rdfs',
      abort_on_first=False,
      allow_infos=False,
      allow_warnings=False,
      meta_shacl=False,
      advanced=False,
      js=False,
      debug=False)
    conforms, results_graph, results_text = r
    return conforms, results_graph, results_text

## Verschillende niveaus van valideren

- De validatie kan kijken naar alleen de verplichte eigenschappen (niveau_1)
- De verplichte eigenschappen, inclusief het bereik van de codelijsten (niveau_2)
- Of verplichte eigenschappen, codelijsten en aanbevolen eigenschappen (niveau_3)

In [2]:
from rdflib import Graph
shacl_folder = r'https://raw.githubusercontent.com/Geonovum/DCAT-AP-NL30/refs/heads/main/shapes/'

dcat_ap = Graph().parse(shacl_folder + 'dcat-ap-SHACL.ttl', format="ttl")
dcat_ap_nl = Graph().parse(shacl_folder + 'dcat-ap-nl-SHACL.ttl', format="ttl")
klassebereik = Graph().parse(shacl_folder + 'dcat-ap-nl-SHACL-klassebereik.ttl', format="ttl")
codelijsten = Graph().parse(shacl_folder + 'dcat-ap-nl-SHACL-klassebereik-codelijsten.ttl', format="ttl")
aanbevolen = Graph().parse(shacl_folder + 'dcat-ap-nl-SHACL-aanbevolen.ttl', format="ttl")

niveau_1 = Graph()
niveau_1 += dcat_ap
niveau_1 += dcat_ap_nl

niveau_2 = Graph()
niveau_2 += niveau_1
niveau_2 += klassebereik
niveau_2 += codelijsten

niveau_3 = Graph()
niveau_3 += niveau_2
niveau_3 += aanbevolen


## Validatie

validatie met pyshacl kan een text file opleveren met leesbare output of een .ttl file met de validatie resultaten.

In onderstaand voorbeeld doen we allebei

In [ ]:
from rdflib import Graph

input_data = r'./dcat-ap-nl-dataset-valide.ttl'

data_graph = Graph()
data_graph.parse(input_data, format='turtle')


# Niveau_1
conforms1, results_graph1, results_text1 = do_validate(data_graph, niveau_1)
results_graph1.serialize('./resultaat/dataset-valide-niveau_1.ttl')
f = open("./resultaat/dataset-valide-niveau_1.txt", "a")
f.write(results_text1)
f.close()

# Niveau_2
conforms1, results_graph1, results_text1 = do_validate(data_graph, niveau_2)
results_graph1.serialize('./resultaat/dataset-valide-niveau_2.ttl')
f = open("./resultaat/dataset-valide-niveau_2.txt", "a")
f.write(results_text1)
f.close()

# Niveau_3
conforms1, results_graph1, results_text1 = do_validate(data_graph, niveau_3)
results_graph1.serialize('./resultaat/dataset-valide-niveau_3.ttl')
f = open("./resultaat/dataset-valide-niveau_3.txt", "a")
f.write(results_text1)
f.close()


Merk op dat niveau_1 en niveau_2 het resultaat
```
Validation Report
Conforms: True
```
oplevert.

niveau_3 geeft 2 warnings van aanbevolen eigenschappen die niet aanwezig zijn.

---

## sparql query op shacl validatie

Als er veel validatie fouten zijn, kan het resultaat nogal onoverzichtelijk zijn.

Met een paar sparql queries kunnen we makkelijk overzicht krijgen


In [10]:
# shacl resultaat en oorspronkelijke data mergen zodat we de dataset labels hebben bij de validatie resultaten.
from rdflib import Graph

validatiegraph = Graph()
validatiegraph.parse(r'./resultaat/dataset-valide-niveau_3.ttl', format='turtle')
validatiegraph += data_graph

validatiegraph.serialize('data_en_resultaat.ttl')


<Graph identifier=N1407fa6d42f7456196cd4c6ab3b482e9 (<class 'rdflib.graph.Graph'>)>

In [11]:
# Define your SPARQL query
sparql_query = """
    PREFIX sh: <http://www.w3.org/ns/shacl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT distinct(count(?path) as ?cnt) ?path ?msg WHERE {
        ?sub a sh:ValidationResult ;
  	    sh:resultPath ?path ;
        sh:resultMessage ?msg .
    } 
    group by  ?path  ?msg
    order by ?cnt 
"""

# Execute the query
results = validatiegraph.query(sparql_query)

# Process the results
for row in results:
    print(row.cnt, row.path, row.msg)

1 http://xmlns.com/foaf/0.1/page Less than 1 values on exampleMS:1T2p3o4B->foaf:page
1 http://www.w3.org/ns/dcat#landingPage Less than 1 values on exampleMS:1T2p3o4B->dcat:landingPage


In [28]:
import pandas as pd
# Define your SPARQL query
sparql_query = """
    PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix dcterms: <http://purl.org/dc/terms/>
SELECT ?title ?path ?msg WHERE {
  ?sub a sh:ValidationResult ;
  	sh:focusNode ?node ;
  	sh:resultPath ?path ;
  	sh:resultMessage ?msg .
  ?node dcterms:title ?title .
  FILTER (LANG(?title) = "nl")
} order by ?title ?path  ?msg
"""
# Execute the query
results = validatiegraph.query(sparql_query)

# present the results in a easy markdown formatted table
results_list = [dict(zip(results.vars, row)) for row in results]
df = pd.DataFrame(results_list)
print(df.to_markdown(index=False, numalign="left", stralign="left"))

| title               | path                                  | msg                                                        |
|:--------------------|:--------------------------------------|:-----------------------------------------------------------|
| Naam van de dataset | http://www.w3.org/ns/dcat#landingPage | Less than 1 values on exampleMS:1T2p3o4B->dcat:landingPage |
| Naam van de dataset | http://xmlns.com/foaf/0.1/page        | Less than 1 values on exampleMS:1T2p3o4B->foaf:page        |
